In [210]:
import numpy as np
from collections import defaultdict
import itertools
from itertools import product

<b> Step 1. Calculate transition and emission probabilities </b>

In [211]:
# Processing training data and create count sets

set_of_tags = set()
set_of_tags.add("<s>"); set_of_tags.add("</s>")
set_of_words = set()

# Get counts of tag, words, word-tags
word_tag_count = defaultdict(int) # C(t, w)
word_count = defaultdict(int) # C(w)
tag_count = defaultdict(int) # C(t)

tag_sequence = ["<s>"]  # Sequence of tags, a sentence ends with .
tag_sequence_count = 0 
tag_tag_count = defaultdict(int) # C(t_{i-1}, t_i)

with open('berp-POS-training.txt', 'r') as train_file:
    train_data = train_file.readlines()

# print('{}'.format(train_data[0][2:]))
    
for line in train_data:
     if line != '\n':
        word_tag = line.strip().split('\t')
        word = word_tag[1]
        set_of_tags.add(word_tag[2])
        set_of_words.add(word_tag[1])
        
        word_tag_count[(word_tag[1], word_tag[2])] += 1
        word_count[word_tag[1]] += 1
        tag_count[word_tag[2]] += 2
        
        # Get all tags from all lines
        tag_sequence.append(word_tag[2])
        if word_tag[1] == ".":
            tag_sequence.append("</s>")
            for i in range(0, len(tag_sequence)-1):
                tag_tag_count[(tag_sequence[i], tag_sequence[i+1])] += 1
            tag_sequence = ["<s>"]
            tag_sequence_count += 1
            
tag_count["<s>"] = tag_sequence_count
tag_count["</s>"] = tag_sequence_count

In [217]:
# Combine tag set and word set and 0 count in word_tag_count if not already there 
all_word_tag_combos = list(product(set_of_words, set_of_tags))
for word_tag_combo in all_word_tag_combos:
    if word_tag_combo not in word_tag_count.keys():
        word_tag_count[word_tag_combo] = 0
        
# Same thing for tag_tag_count
all_tag_tag_combos = list(product(set_of_tags, set_of_tags))
for tag_tag_combo in all_tag_tag_combos:
    if tag_tag_combo not in tag_tag_count.keys():
        tag_tag_count[tag_tag_combo] = 0

In [218]:
def calculate_transition(tag_tag_count, tag_count):
    # Calculate transition probabilities
    # P(t_i | t_{i-1}) = C(t_{i-1, t_i}) / C(t_{i-1})
    transition_probs = {}
    for tag_tag in tag_tag_count:
        count_tag1tag2 = tag_tag_count[tag_tag]
        count_tag1 = tag_count[tag_tag[0]]
        transition_probs[tag_tag] = count_tag1tag2 / count_tag1        
    return transition_probs

In [219]:
def calculate_emission(word_tag_count, tag_count):
    # Calculate emission probabilities
    # P(w | t) = C(t, w) / C(t)
    # word-tag pairings with 0 prob not included, will be 0 by default if not in dict
    emission_probs = {}
    for word_tag in word_tag_count:
        count_word_tag = word_tag_count[word_tag]
        count_tag = tag_count[word_tag[1]]
        emission_probs[word_tag] = count_word_tag / count_tag
    return emission_probs

In [293]:
transition_probs = calculate_transition(tag_tag_count, tag_count)
emission_probs = calculate_emission(word_tag_count, tag_count)
list_of_tags = list(sorted(set_of_tags))
list_of_words = list(sorted(set_of_words))

<b><i> To do later: Smoothing, dealing with unseen </i></b>

In [312]:
transition_probs

{('<s>', 'PRP'): 0.34810286146476743,
 ('PRP', 'MD'): 0.13616002471424157,
 ('MD', 'VB'): 0.39798183652875885,
 ('VB', 'TO'): 0.08662310866574965,
 ('TO', 'VB'): 0.4697436885719792,
 ('VB', 'IN'): 0.07200825309491059,
 ('IN', 'DT'): 0.10410546139359698,
 ('DT', 'JJ'): 0.11639382329037501,
 ('JJ', 'NN'): 0.2820465459918729,
 ('NN', '.'): 0.17129925027915138,
 ('.', '</s>'): 0.5,
 ('VB', 'JJ'): 0.02764786795048143,
 ('<s>', 'JJ'): 0.02262700113450145,
 ('<s>', 'NN'): 0.04922475734274549,
 ('PRP', 'VBP'): 0.1611059623107816,
 ('VBP', 'TO'): 0.12834539639791281,
 ('VB', 'NN'): 0.03978679504814305,
 ('VB', 'RB'): 0.0234525447042641,
 ('RB', 'JJ'): 0.07445036642238508,
 ('JJ', '.'): 0.046053441694372615,
 ('<s>', 'RB'): 0.03939241144585907,
 ('RB', 'RB'): 0.06054297135243171,
 ('RB', 'IN'): 0.05754497001998667,
 ('IN', 'PRP'): 0.01864406779661017,
 ('VB', '.'): 0.01186382393397524,
 ('IN', 'NN'): 0.16666666666666666,
 ('<s>', 'VB'): 0.11704273288793647,
 ('VB', 'PRP'): 0.08648555708390647,
 

In [313]:
emission_probs

{('i', 'PRP'): 0.30525949953660797,
 ("'d", 'MD'): 0.12734611503531787,
 ('like', 'VB'): 0.09305364511691884,
 ('to', 'TO'): 0.46463673154750434,
 ('go', 'VB'): 0.039133425034387895,
 ('to', 'IN'): 0.03472693032015066,
 ('a', 'DT'): 0.1496574944850807,
 ('fancy', 'JJ'): 0.0005541189508681197,
 ('restaurant', 'NN'): 0.03722683043547615,
 ('.', '.'): 0.5,
 ('french', 'JJ'): 0.011328654106637113,
 ('food', 'NN'): 0.04883155208167172,
 ('next', 'JJ'): 0.006095308459549317,
 ('thursday', 'NN'): 0.0017746052001914182,
 ('dinner', 'NN'): 0.018144839687350454,
 ('want', 'VBP'): 0.1512371654603602,
 ('eat', 'VB'): 0.04886519944979367,
 ('have', 'VB'): 0.042090784044016505,
 ('it', 'PRP'): 0.037805066419524254,
 ('can', 'MD'): 0.09616548940464177,
 ('be', 'VB'): 0.03352819807427786,
 ('really', 'RB'): 0.012075283144570287,
 ('expensive', 'JJ'): 0.01686984361531831,
 ('as', 'RB'): 0.004413724183877415,
 ('far', 'RB'): 0.01615589606928714,
 ('away', 'RB'): 0.0181545636242505,
 ('as', 'IN'): 0.0024

<b> Step 2. Viterbi algorithm </b>

In [438]:
# Initializing matrices & vector
# Matrix with tags vs words
p = np.zeros(shape=(len(list_of_tags), len(input_sentence)))
# Matrix for backtrace
back = np.zeros(shape=(len(list_of_tags), len(input_sentence)), dtype=np.int)

# Initializing step: P(tag|start) * P(word1|tag)
for tag_i, tag in enumerate(list_of_tags):
    # Fill first col of matrix p & back matrices
    tag_given_start = ('<s>', tag)
    word_given_tag = (input_sentence[0], tag)
    p[tag_i, 0] = transition_probs[tag_given_start] * emission_probs[word_given_tag]
    back[tag_i, 0] = 0  # RECHECK this - not sure how to initialize back pointer
    
# Recursion step - go through every tag for each token:
for word_i in range(1, len(input_sentence)):
    for tagi1, tag1 in enumerate(list_of_tags):
        # For each tag, get its prob given all other tags:
        # Prev column * P(tag|all tags) * P(word|tag)
#         for tag_i2, tag2 in enumerate(set_of_tags):
#             # Tag2 prob at previous column
#             term1 = p[tag_i2, word_i - 1]
                        
#             # P(current tag|all tags):
#             tag_given_tag = (tag1, tag2)
#             term2 = transition_probs[tag_given_tag]
            
#             # P(word|current_tag):
#             word_given_tag = (input_sentence[word_i], tag1)
#             term3 = emission_probs[word_given_tag]
            
#             # Multiplied together and get max 
#             p[tag_i1, word_i] = np.max(term1 * term2 * term3)
#             back[tag_i1, word_i] = np.argmax(term1 * term2)

        # Fill in viterbi matrix
        p[tagi1, word_i] = np.max([p[tagi2, word_i - 1] * transition_probs[tag2, tag1] * emission_probs[input_sentence[word_i], tag1] for tagi2,tag2 in enumerate(list_of_tags)])
        # Fill in backpointer
        back[tagi1, word_i] = np.argmax([p[tagi2, word_i - 1] * transition_probs[tag2, tag1] * emission_probs[input_sentence[word_i], tag1] for tagi2,tag2 in enumerate(list_of_tags)])
        
# Termination steps
best_path_prob = np.max([p[tag_i, len(input_sentence)-1] for tag_i, tag in enumerate(list_of_tags)])        
best_path_pointer = np.argmax([p[tag_i, len(input_sentence)-1] for tag_i, tag in enumerate(list_of_tags)])        
best_path_pointer

0

In [468]:
path_idx = [best_path_pointer]
for column_i, column in enumerate(back.T[::-1]): # Starts at end 
    max_tag_idx = max(column)
    path_idx.append(max_tag_idx)
# print(path_idx)
    
tag_seq = []
for i in range(0,len(path_idx)-1):
    tag_seq.append(list_of_tags[path_idx[i]])

print(input_sentence)
tag_seq[::-1]


['i', "'d", 'like', 'to', 'go', 'to', 'a', 'fancy', 'restaurant', '.']


['PRP', 'MD', 'VB', 'TO', 'VB', 'IN', 'DT', 'JJ', 'NN', '.']

In [437]:
input_sentence = ['i', "'d", 'like', 'to', 'go', 'to', 'a', 'fancy', 'restaurant', '.']